In [1]:
import pandas as pd
import os
import requests
import time
import random

from io import BytesIO
from PIL import Image
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Initialize edge browser
edge_option = webdriver.EdgeOptions()
edge_option.add_argument('--no-sandbox')
driver = webdriver.Edge(options=edge_option)

# Create a folder for storing articles
root_dir = './vn_news_thumbnail'
os.makedirs(root_dir, exist_ok=True)
n_pages = 10 # Change if you want more articles
img_id = 0

for page_idx in tqdm(range(n_pages)):
    # Access to table page
    main_url = f'https://vietnamnet.vn/thoi-su-page{page_idx}' 
    driver.get(main_url)

    # Get list of images (list of tags)
    imgs_lst_xpath = '//div[@class="topStory-15nd"]/div/div[1]/a/img'
    imgs_tags = driver.find_elements(
        By.XPATH, 
        imgs_lst_xpath
    )
    # Get list of image urls
    img_urls = [imgs_tag.get_attribute('src') for imgs_tag in imgs_tags]

    for img_url in img_urls:
        # Get image information
        img_url_resp = requests.get(img_url)
        try:
            # Read image
            img = Image.open(
                BytesIO(img_url_resp.content)
            )
        except:
            continue

        # Convert image to RGB format if needed
        if img.mode == 'P':
            img = img.convert('RGB') 

        # Save crawled image
        img_name = f"IMG_{img_id:05}.png"
        img_save_path = os.path.join(root_dir, img_name)
        img.save(img_save_path) # Read how to use this method: https://www.geeksforgeeks.org/python-pil-image-save-method/
        img_id += 1

100%|██████████| 10/10 [01:46<00:00, 10.65s/it]
